In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
from datetime import date
import pickle
import requests
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import tqdm
import sys
sys.path.append("../")
import src.soporte as sp

c:\Users\Lin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
df = pd.read_csv("../data/datamarket_productos_de_supermercados.csv")

In [8]:
df.shape

(7843438, 10)

In [12]:
# No me sirve este dato y hay muchos nulos
df.drop(["description"], axis = 1, inplace = True)

In [13]:
dfm = df[df["supermarket"] == "mercadona-es"]
dfc = df[df["supermarket"] == "carrefour-es"]
dfd = df[df["supermarket"] == "dia-es"]

In [14]:
# hay una categoria rara, la drop
dfd2 = dfd[dfd["category"] != "soy_solidario_soy_solidario_soy_solidario"]

In [15]:
dfd2["subcategoria"] = dfd2.apply(lambda x: sp.dia_subcat(x["category"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_67684\174307093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd2["subcategoria"] = dfd2.apply(lambda x: sp.dia_subcat(x["category"]), axis = 1)


In [16]:
dfd2["category"] = dfd2.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_67684\1595198542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd2["category"] = dfd2.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)


In [17]:
dfm["subcategoria"] = dfm.apply(lambda x: sp.mer_subcat(x["category"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_67684\1970358374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm["subcategoria"] = dfm.apply(lambda x: sp.mer_subcat(x["category"]), axis = 1)


In [18]:
dfm["categoria"] = dfm.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)

C:\Users\Lin\AppData\Local\Temp\ipykernel_67684\569603170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm["categoria"] = dfm.apply(lambda x: sp.category(x["subcategoria"]), axis = 1)
